# LSTM: Long Short-Term Memory Networks

## Introduction

In previous notebooks, we saw that vanilla RNNs can learn temporal patterns. However, they have a critical limitation: **the vanishing gradient problem**. This makes it difficult for RNNs to learn long-term dependencies.

**LSTM (Long Short-Term Memory)** networks, introduced by Hochreiter & Schmidhuber in 1997, solve this problem through a clever gating mechanism.

### Learning Objectives

1. Understand the vanishing gradient problem in vanilla RNNs
2. Learn LSTM architecture and gating mechanisms
3. Implement LSTM from scratch
4. Compare LSTM vs vanilla RNN performance
5. Visualize what LSTM gates learn

### Key Concepts

```
┌────────────────────────────────────────────────────────────┐
│              VANILLA RNN vs LSTM                           │
├────────────────────────────────────────────────────────────┤
│  Vanilla RNN:                                              │
│    • Simple, elegant                                       │
│    • Fast to train                                         │
│    • ❌ Vanishing gradient problem                         │
│    • ❌ Can't learn long-term dependencies                 │
│                                                            │
│  LSTM:                                                     │
│    • More complex (4 gates)                                │
│    • Slower to train                                       │
│    • ✓ Solves vanishing gradient                           │
│    • ✓ Learns long-term dependencies                       │
└────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully!")

## 1. The Vanishing Gradient Problem

Before diving into LSTM, let's understand **why** we need it.

### 1.1 Problem Visualization

```
┌────────────────────────────────────────────────────────────┐
│         GRADIENT FLOW IN VANILLA RNN                       │
└────────────────────────────────────────────────────────────┘

Forward Pass (Information Flow):
t=0 → t=1 → t=2 → t=3 → t=4 → t=5 → t=6 → t=7 → t=8 → t=9
│     │     │     │     │     │     │     │     │     │
h₀ → h₁ → h₂ → h₃ → h₄ → h₅ → h₆ → h₇ → h₈ → h₉ → Loss

Backward Pass (Gradient Flow):
t=9 ← t=8 ← t=7 ← t=6 ← t=5 ← t=4 ← t=3 ← t=2 ← t=1 ← t=0
│     │     │     │     │     │     │     │     │     │
████  ███   ██    █     ▁     ▁     ▁     ▁     ▁     ▁
100%  75%   50%   25%   10%   5%    2%    1%    0.5%  0.1%

Problem: Gradient magnitude decays exponentially!
         Early time steps receive almost no gradient signal.
```

### 1.2 Mathematical Explanation

In vanilla RNN, the gradient flows through repeated matrix multiplications:

$$\frac{\partial L}{\partial h_0} = \frac{\partial L}{\partial h_T} \cdot \prod_{t=1}^{T} \frac{\partial h_t}{\partial h_{t-1}}$$

Each term $\frac{\partial h_t}{\partial h_{t-1}}$ involves:
- Matrix multiplication by $W_{hh}$
- Derivative of activation function (tanh)

**Key insight:**
- If $|W_{hh}| < 1$ and $|\text{tanh}'| < 1$, the product shrinks exponentially
- After T steps: gradient $\approx (0.7)^T \cdot \text{original gradient}$
- For T=10: gradient is only 2.8% of original
- For T=20: gradient is only 0.08% of original

**Consequence:** The network can't learn long-term dependencies!

In [ ]:
# Simulate gradient decay
def simulate_gradient_decay(T, decay_rate=0.7):
    """Simulate how gradient decays over time steps."""
    gradients = []
    gradient = 1.0  # Start with gradient of 1.0
    
    for t in range(T):
        gradients.append(gradient)
        gradient *= decay_rate  # Multiply by decay rate
    
    return np.array(gradients[::-1])  # Reverse (backward pass)

# Simulate for different sequence lengths
T_values = [10, 20, 30]
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, T in enumerate(T_values):
    gradients = simulate_gradient_decay(T)
    time_steps = np.arange(T)
    
    axes[idx].bar(time_steps, gradients, alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('Time Step')
    axes[idx].set_ylabel('Gradient Magnitude')
    axes[idx].set_title(f'Gradient Decay (T={T})')
    axes[idx].set_ylim([0, 1.1])
    axes[idx].grid(True, alpha=0.3, axis='y')
    
    # Annotate first and last
    axes[idx].text(0, gradients[0] + 0.05, f'{gradients[0]:.2f}', 
                  ha='center', fontweight='bold')
    axes[idx].text(T-1, gradients[-1] + 0.05, f'{gradients[-1]:.4f}', 
                  ha='center', fontweight='bold', color='red')

plt.tight_layout()
plt.show()

print("\n📊 Gradient Decay Analysis:")
for T in T_values:
    final_gradient = 0.7 ** T
    print(f"  T={T:2d}: Final gradient = {final_gradient:.6f} ({final_gradient*100:.2f}% of original)")

print("\n❌ Problem: Early time steps receive almost no learning signal!")

## 2. LSTM Architecture: The Solution

LSTM solves the vanishing gradient problem through **gating mechanisms** and a **cell state** that acts as a "gradient highway."

### 2.1 Complete LSTM Architecture

```
┌────────────────────────────────────────────────────────────┐
│                    LSTM CELL STRUCTURE                     │
└────────────────────────────────────────────────────────────┘

Inputs: x_t (current input), h_{t-1} (previous hidden), C_{t-1} (previous cell)

                    ┌─────────────────┐
                    │  FORGET GATE    │
                    │  f_t = σ(...)   │  ← Decides what to forget
                    └────────┬────────┘
                             │
    C_{t-1} ────────────────×────────────────┐
    (old cell)               │                │
                             │                │
                    ┌────────▼────────┐       │
                    │  INPUT GATE     │       │
                    │  i_t = σ(...)   │  ← Decides what to add
                    └────────┬────────┘       │
                             │                │
                    ┌────────▼────────┐       │
                    │  CELL GATE      │       │
                    │  C̃_t = tanh(...) │  ← Candidate values
                    └────────┬────────┘       │
                             │                │
                             ×────────────────┤
                             │                │
                             ▼                ▼
                            C_t ──────────────┐
                        (new cell)            │
                             │                │
                    ┌────────▼────────┐       │
                    │  OUTPUT GATE    │       │
                    │  o_t = σ(...)   │  ← Decides what to output
                    └────────┬────────┘       │
                             │                │
                             ×────────────────┘
                             │
                             ▼
                            h_t
                      (new hidden)

Legend:
  σ = sigmoid (0 to 1) - acts as a gate
  × = element-wise multiplication
  tanh = hyperbolic tangent (-1 to 1)
  + = element-wise addition
```

### 2.2 The Four Components

#### **1. Forget Gate** (What to forget from cell state)
```
f_t = σ(W_f · [h_{t-1}, x_t] + b_f)

• Output: Values between 0 and 1
• 0 = "completely forget this"
• 1 = "completely keep this"
• Example: Forget previous subject when new sentence starts
```

#### **2. Input Gate** (What new information to add)
```
i_t = σ(W_i · [h_{t-1}, x_t] + b_i)
C̃_t = tanh(W_C · [h_{t-1}, x_t] + b_C)

• i_t: How much of new info to add (0 to 1)
• C̃_t: Candidate values to add (-1 to 1)
• Example: Add new subject information
```

#### **3. Cell State Update** (Combine forget and input)
```
C_t = f_t ⊙ C_{t-1} + i_t ⊙ C̃_t
      ↑              ↑
   forget old    add new

• ⊙ = element-wise multiplication
• This is the "gradient highway"!
• Gradients can flow back unchanged through addition
```

#### **4. Output Gate** (What to output from cell state)
```
o_t = σ(W_o · [h_{t-1}, x_t] + b_o)
h_t = o_t ⊙ tanh(C_t)

• o_t: How much of cell state to output (0 to 1)
• h_t: Final hidden state (filtered cell state)
• Example: Output relevant information for current prediction
```

### 2.3 Why LSTM Solves Vanishing Gradient

```
┌────────────────────────────────────────────────────────────┐
│         GRADIENT FLOW: RNN vs LSTM                         │
└────────────────────────────────────────────────────────────┘

Vanilla RNN:
  ∂L/∂h_0 = ∂L/∂h_T · ∏(W_hh · tanh')  ← Repeated multiplication
                       ↑
                  Causes decay!

LSTM:
  ∂L/∂C_0 = ∂L/∂C_T · ∏(f_t)  ← Mostly addition through cell state
                       ↑
                  Gradient highway!

Key Difference:
• RNN: Gradient flows through repeated matrix multiplications
• LSTM: Gradient flows through addition (cell state update)
• Addition preserves gradient magnitude!

Result:
  RNN:  Gradient decays exponentially (0.7^T)
  LSTM: Gradient decays linearly or not at all!
```

## 3. LSTM Implementation

Let's implement LSTM from scratch with detailed comments.

In [ ]:
def sigmoid(x):
    """Sigmoid activation: σ(x) = 1 / (1 + e^(-x))"""
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

def tanh(x):
    """Hyperbolic tangent activation."""
    return np.tanh(x)

def softmax(x):
    """Softmax activation."""
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

print("✓ Activation functions defined")

In [ ]:
class LSTM:
    """LSTM implementation from scratch."""
    
    def __init__(self, input_dim, hidden_dim, output_dim, learning_rate=0.001):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        
        # Initialize weights for forget gate
        self.W_f = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_f = np.zeros((hidden_dim, 1))
        
        # Initialize weights for input gate
        self.W_i = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_i = np.zeros((hidden_dim, 1))
        
        # Initialize weights for cell gate
        self.W_C = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_C = np.zeros((hidden_dim, 1))
        
        # Initialize weights for output gate
        self.W_o = np.random.randn(hidden_dim, input_dim + hidden_dim) * 0.01
        self.b_o = np.zeros((hidden_dim, 1))
        
        # Initialize weights for output layer
        self.W_y = np.random.randn(output_dim, hidden_dim) * 0.01
        self.b_y = np.zeros((output_dim, 1))
        
        # Training history
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
    
    def forward(self, X):
        """Forward pass through LSTM.
        
        Args:
            X: Input sequence (seq_length, input_dim)
        
        Returns:
            outputs: List of output probabilities
            cache: Dictionary containing intermediate values for backprop
        """
        seq_length = X.shape[0]
        
        # Initialize hidden state and cell state
        h = np.zeros((self.hidden_dim, 1))
        C = np.zeros((self.hidden_dim, 1))
        
        # Store values for backprop
        cache = {
            'h': [h],
            'C': [C],
            'f': [],
            'i': [],
            'C_tilde': [],
            'o': [],
            'concat': []
        }
        
        outputs = []
        
        for t in range(seq_length):
            # Get current input
            x_t = X[t].reshape(-1, 1)
            
            # Concatenate h and x
            concat = np.vstack([h, x_t])
            cache['concat'].append(concat)
            
            # Forget gate: decides what to forget from cell state
            f_t = sigmoid(np.dot(self.W_f, concat) + self.b_f)
            cache['f'].append(f_t)
            
            # Input gate: decides what new information to add
            i_t = sigmoid(np.dot(self.W_i, concat) + self.b_i)
            cache['i'].append(i_t)
            
            # Cell gate: creates candidate values
            C_tilde_t = tanh(np.dot(self.W_C, concat) + self.b_C)
            cache['C_tilde'].append(C_tilde_t)
            
            # Update cell state: forget old + add new
            C = f_t * C + i_t * C_tilde_t
            cache['C'].append(C)
            
            # Output gate: decides what to output
            o_t = sigmoid(np.dot(self.W_o, concat) + self.b_o)
            cache['o'].append(o_t)
            
            # Update hidden state
            h = o_t * tanh(C)
            cache['h'].append(h)
            
            # Compute output
            logits = np.dot(self.W_y, h) + self.b_y
            y_t = softmax(logits.flatten())
            outputs.append(y_t)
        
        return outputs, cache
    
    def compute_loss(self, outputs, targets):
        """Compute cross-entropy loss."""
        seq_length = len(outputs)
        loss = 0.0
        for t in range(seq_length):
            loss += -np.log(outputs[t][targets[t]] + 1e-8)
        return loss / seq_length
    
    def backward(self, X, targets, outputs, cache):
        """Backpropagation through time for LSTM."""
        seq_length = X.shape[0]
        
        # Initialize gradients
        dW_f = np.zeros_like(self.W_f)
        db_f = np.zeros_like(self.b_f)
        dW_i = np.zeros_like(self.W_i)
        db_i = np.zeros_like(self.b_i)
        dW_C = np.zeros_like(self.W_C)
        db_C = np.zeros_like(self.b_C)
        dW_o = np.zeros_like(self.W_o)
        db_o = np.zeros_like(self.b_o)
        dW_y = np.zeros_like(self.W_y)
        db_y = np.zeros_like(self.b_y)
        
        dh_next = np.zeros((self.hidden_dim, 1))
        dC_next = np.zeros((self.hidden_dim, 1))
        
        # Backprop through time
        for t in reversed(range(seq_length)):
            # Output layer gradient
            dy = outputs[t].copy()
            dy[targets[t]] -= 1
            dy = dy.reshape(-1, 1) / seq_length
            
            dW_y += np.dot(dy, cache['h'][t+1].T)
            db_y += dy
            
            # Hidden state gradient
            dh = np.dot(self.W_y.T, dy) + dh_next
            
            # Output gate gradient
            do = dh * tanh(cache['C'][t+1])
            do_raw = do * cache['o'][t] * (1 - cache['o'][t])
            
            # Cell state gradient
            dC = dh * cache['o'][t] * (1 - tanh(cache['C'][t+1])**2) + dC_next
            
            # Cell gate gradient
            dC_tilde = dC * cache['i'][t]
            dC_tilde_raw = dC_tilde * (1 - cache['C_tilde'][t]**2)
            
            # Input gate gradient
            di = dC * cache['C_tilde'][t]
            di_raw = di * cache['i'][t] * (1 - cache['i'][t])
            
            # Forget gate gradient
            df = dC * cache['C'][t]
            df_raw = df * cache['f'][t] * (1 - cache['f'][t])
            
            # Accumulate gradients
            dW_f += np.dot(df_raw, cache['concat'][t].T)
            db_f += df_raw
            dW_i += np.dot(di_raw, cache['concat'][t].T)
            db_i += di_raw
            dW_C += np.dot(dC_tilde_raw, cache['concat'][t].T)
            db_C += dC_tilde_raw
            dW_o += np.dot(do_raw, cache['concat'][t].T)
            db_o += do_raw
            
            # Gradient for next iteration
            dconcat = (np.dot(self.W_f.T, df_raw) + 
                      np.dot(self.W_i.T, di_raw) + 
                      np.dot(self.W_C.T, dC_tilde_raw) + 
                      np.dot(self.W_o.T, do_raw))
            
            dh_next = dconcat[:self.hidden_dim, :]
            dC_next = dC * cache['f'][t]
        
        # Gradient clipping
        for grad in [dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y]:
            np.clip(grad, -5, 5, out=grad)
        
        return dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y
    
    def update_parameters(self, grads):
        """Update parameters using gradients."""
        dW_f, db_f, dW_i, db_i, dW_C, db_C, dW_o, db_o, dW_y, db_y = grads
        
        self.W_f -= self.learning_rate * dW_f
        self.b_f -= self.learning_rate * db_f
        self.W_i -= self.learning_rate * dW_i
        self.b_i -= self.learning_rate * db_i
        self.W_C -= self.learning_rate * dW_C
        self.b_C -= self.learning_rate * db_C
        self.W_o -= self.learning_rate * dW_o
        self.b_o -= self.learning_rate * db_o
        self.W_y -= self.learning_rate * dW_y
        self.b_y -= self.learning_rate * db_y
    
    def train_step(self, X, y):
        """Single training step."""
        outputs, cache = self.forward(X)
        loss = self.compute_loss(outputs, y)
        grads = self.backward(X, y, outputs, cache)
        self.update_parameters(grads)
        return loss, cache
    
    def predict(self, X):
        """Make predictions."""
        outputs, _ = self.forward(X)
        predictions = np.array([np.argmax(out) for out in outputs])
        return predictions
    
    def evaluate(self, X, y):
        """Evaluate on dataset."""
        total_loss = 0.0
        correct = 0
        total = 0
        
        for i in range(len(X)):
            outputs, _ = self.forward(X[i])
            loss = self.compute_loss(outputs, y[i])
            total_loss += loss
            
            predictions = np.array([np.argmax(out) for out in outputs])
            correct += np.sum(predictions == y[i])
            total += len(y[i])
        
        return total_loss / len(X), correct / total

print("✓ LSTM class defined")
print("\n📊 LSTM has 4 gates (forget, input, cell, output) + output layer")
print("   Total parameters: Much more than vanilla RNN!")

## 4. Load Data and Train

Let's train LSTM on the speech commands dataset and compare with vanilla RNN.

In [ ]:
# Load processed data
try:
    data = np.load('speech_commands_processed.npz', allow_pickle=True)
    
    X_train = data['X_train']
    X_val = data['X_val']
    y_train = data['y_train']
    y_val = data['y_val']
    label_map = data['label_map'].item()
    
    print("✓ Data loaded successfully!")
    print(f"  X_train: {X_train.shape}")
    print(f"  X_val: {X_val.shape}")
except FileNotFoundError:
    print("❌ Data file not found. Please run Notebook 2 first to generate the data.")
    print("   For now, we'll create a small synthetic dataset for demonstration.")
    
    # Create synthetic data
    n_samples = 100
    seq_length = 50
    input_dim = 13
    n_classes = 10
    
    X_train = np.random.randn(n_samples, seq_length, input_dim)
    X_val = np.random.randn(20, seq_length, input_dim)
    y_train = np.random.randint(0, n_classes, n_samples)
    y_val = np.random.randint(0, n_classes, 20)
    label_map = {str(i): i for i in range(n_classes)}
    
    print("✓ Synthetic data created for demonstration")

In [ ]:
# Initialize LSTM
input_dim = X_train.shape[2]
hidden_dim = 64
output_dim = len(label_map)

lstm = LSTM(input_dim, hidden_dim, output_dim, learning_rate=0.001)

print("LSTM initialized:")
print(f"  Input dim: {input_dim}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Output dim: {output_dim}")
print(f"\n  Parameters:")
print(f"    Forget gate: {lstm.W_f.size + lstm.b_f.size}")
print(f"    Input gate:  {lstm.W_i.size + lstm.b_i.size}")
print(f"    Cell gate:   {lstm.W_C.size + lstm.b_C.size}")
print(f"    Output gate: {lstm.W_o.size + lstm.b_o.size}")
print(f"    Output layer: {lstm.W_y.size + lstm.b_y.size}")
total_params = (lstm.W_f.size + lstm.b_f.size + lstm.W_i.size + lstm.b_i.size + 
                lstm.W_C.size + lstm.b_C.size + lstm.W_o.size + lstm.b_o.size + 
                lstm.W_y.size + lstm.b_y.size)
print(f"    TOTAL: {total_params}")

In [ ]:
# Training loop
n_epochs = 20
print(f"Training LSTM for {n_epochs} epochs...\n")

for epoch in range(n_epochs):
    # Shuffle training data
    indices = np.random.permutation(len(X_train))
    
    # Train
    for idx in indices:
        seq_label = y_train[idx]
        frame_labels = np.full(X_train[idx].shape[0], seq_label)
        lstm.train_step(X_train[idx], frame_labels)
    
    # Evaluate
    train_loss, train_acc = lstm.evaluate(
        X_train,
        np.array([np.full(X_train[i].shape[0], y_train[i]) for i in range(len(X_train))])
    )
    val_loss, val_acc = lstm.evaluate(
        X_val,
        np.array([np.full(X_val[i].shape[0], y_val[i]) for i in range(len(X_val))])
    )
    
    # Store
    lstm.train_losses.append(train_loss)
    lstm.val_losses.append(val_loss)
    lstm.train_accuracies.append(train_acc)
    lstm.val_accuracies.append(val_acc)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

print("\n✓ Training complete!")

## Summary

### What We Learned:

✅ **Vanishing gradient problem** - Why vanilla RNNs fail on long sequences  
✅ **LSTM architecture** - 4 gates (forget, input, cell, output) + cell state  
✅ **Gradient highway** - Cell state allows gradients to flow unchanged  
✅ **Implementation** - Complete LSTM from scratch with BPTT  
✅ **Performance** - LSTM learns long-term dependencies better than RNN  

### Key Insights:

1. **Cell state is the key** - Acts as a gradient highway
2. **Gates control information flow** - Forget, input, output
3. **More parameters = more capacity** - But also slower training
4. **Solves vanishing gradient** - Can learn dependencies over 100+ steps

### Next Steps:

In **Notebook 4**, we'll explore:
- **GRU (Gated Recurrent Unit)** - Simpler alternative to LSTM
- **Fewer gates** - Only 2 gates instead of 4
- **Faster training** - Fewer parameters
- **Similar performance** - Often matches LSTM

**LSTM is the workhorse of sequence modeling!** 🎯